In [ ]:
## Drive mount

from google.colab import drive
drive.mount('/content/drive')
!cd drive/MyDrive/ColabNotebooks  && ls

Mounted at /content/drive
/bin/bash: line 0: cd: drive/MyDrive/ColabNotebooks: No such file or directory


In [ ]:
!cd "/content/drive/MyDrive/Colab Notebooks"  && ls

datasets  helpers  Train_car_person.ipynb  yolov5


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
cwd = os.getcwd()
print(cwd)

/content/drive/MyDrive/Colab Notebooks


In [ ]:
parent_directory = ''



root_directory = '/home/shalini'


parent_directory = '/content/drive/MyDrive/Colab Notebooks'



datasets_directory = parent_directory+'/datasets'


dataset_name = 'car-person'
yoloData_directory = datasets_directory+'/'+dataset_name

yoloData_images_directory = yoloData_directory+'/images'
yoloData_label_directory = yoloData_directory+'/labels'

yoloData_images_train_directory = yoloData_images_directory+'/train'
yoloData_images_val_directory = yoloData_images_directory+'/val'

yoloData_labels_train_directory = yoloData_label_directory+'/train'
yoloData_labels_val_directory = yoloData_label_directory+'/val'

yolo_image_size = 640


DATA_LINK = 'https://evp-ml-data.s3.us-east-2.amazonaws.com/ml-interview/openimages-personcar/trainval.tar.gz'

DATA_FILE_NAME = 'cars_persons'


TARGET_DOWNLOAD_FILEPATH = datasets_directory+'/'+DATA_FILE_NAME+'.tar.gz'

unzipped_data_directory = datasets_directory+'/'+DATA_FILE_NAME+'_unzipped'

all_images_directory = unzipped_data_directory+'/trainval/images'

In [ ]:
unzipped_data_directory

'/content/drive/MyDrive/Colab Notebooks/datasets/cars_persons_unzipped'

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10923, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10923 (delta 1), reused 5 (delta 1), pack-reused 10916
Receiving objects: 100% (10923/10923), 11.07 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (7540/7540), done.


In [ ]:
import urllib.request

#%%
if not os.path.exists(datasets_directory):
    os.makedirs(datasets_directory)

urllib.request.urlretrieve(DATA_LINK,TARGET_DOWNLOAD_FILEPATH)
  

('/content/drive/MyDrive/Colab Notebooks/datasets/cars_persons.tar.gz',
 <http.client.HTTPMessage at 0x7f7eedd47550>)

In [ ]:
import tarfile
  
# open file
with tarfile.open(TARGET_DOWNLOAD_FILEPATH)
  
# extracting file
file.extractall(unzipped_data_directory)
  
file.close()

In [ ]:
import json
import pandas as pd
import os

import random

from PIL import Image

from tqdm import tqdm

#%%

def create_bb_coords(x):
    top_left_x,top_left_y,width,height = x
    x1 = top_left_x
    y1 = top_left_y
    x2 = top_left_x+width
    y2 = top_left_y+height
    return [x1,y1,x2,y2]

def return_yolo_centre_width_height(x):
    top_left_x,top_left_y,width,height = x
    x1 = top_left_x
    y1 = top_left_y
    x2 = top_left_x+width
    y2 = top_left_y+height
    return {'centre_x':int((x1+x2)/2),
            'centre_y':int((y1+y2)/2),
            'width':width,
            'height':height}


In [ ]:
def create_yolo_data(train_images_df,labels_df,yoloData_images_train_directory,yoloData_labels_train_directory):
    
    for i in tqdm(range(len(train_images_df))):
        
        #% RESIZE and SAVE IMAGE
        image_file_name = train_images_df['file_name'][i]
        img = Image.open(all_images_directory+'/'+image_file_name)
        actual_width,actual_height = img.width,img.height
        img = img.resize([yolo_image_size,yolo_image_size])
        img.save(yoloData_images_train_directory+'/'+image_file_name)
        
        image_labels_df = labels_df[labels_df['image_id']==train_images_df['image_id'][i]].reset_index(drop=True)
        
        image_labels_df['centre_x'] /= actual_width
        image_labels_df['centre_y'] /= actual_height
        image_labels_df['width'] /= actual_width
        image_labels_df['height'] /= actual_height
        
               
        image_labels_df.drop(['image_id'],axis=1,inplace=True)
        
        # SAVE THE LABEL FILE
        labels_file_name = image_file_name.replace('jpg','txt')
        image_labels_df.to_csv(yoloData_labels_train_directory+'/'+labels_file_name,header=False,index=False,sep=' ')

In [ ]:
with open(unzipped_data_directory+'/trainval/annotations/bbox-annotations.json') as json_file:
    data = json.load(json_file)
  
#%%

labels_df = pd.DataFrame(data['annotations'])

labels_df['yolov5_coords'] = labels_df['bbox'].apply(return_yolo_centre_width_height)

labels_df = pd.concat([labels_df.drop(['yolov5_coords'], axis=1), labels_df['yolov5_coords'].apply(pd.Series)], axis=1)

labels_df = labels_df[['image_id','category_id','centre_x','centre_y','width','height']]

labels_df['category_id'] = labels_df['category_id'].replace({2:0})

images_df = pd.DataFrame(data['images']).rename({'id':'image_id'},axis=1)



In [ ]:
# if not os.path.exists(yoloData_all_images_directory):
#     os.makedirs(yoloData_all_images_directory)
    
if not os.path.exists(yoloData_images_train_directory):
    os.makedirs(yoloData_images_train_directory)

if not os.path.exists(yoloData_images_val_directory):
    os.makedirs(yoloData_images_val_directory)
    
if not os.path.exists(yoloData_labels_train_directory):
    os.makedirs(yoloData_labels_train_directory)

if not os.path.exists(yoloData_labels_val_directory):
    os.makedirs(yoloData_labels_val_directory)    
    

In [ ]:
random_test_indices = random.sample(range(len(images_df)),100)

train_images_df = images_df[~images_df.index.isin(random_test_indices)].reset_index()

val_images_df = images_df[images_df.index.isin(random_test_indices)].reset_index()


In [ ]:
create_yolo_data(train_images_df,labels_df,yoloData_images_train_directory,yoloData_labels_train_directory)

100%|██████████| 2139/2139 [10:00<00:00,  3.56it/s]


In [ ]:
create_yolo_data(val_images_df,labels_df,yoloData_images_val_directory,yoloData_labels_val_directory) 

100%|██████████| 100/100 [00:28<00:00,  3.46it/s]


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/yolov5')
cwd = os.getcwd()
print(cwd)

/content/drive/MyDrive/Colab Notebooks/yolov5


In [ ]:
!python3 train.py --img 640 --batch 16 --epochs 10 --data car-person.yaml --weights yolov5l.pt

train: weights=yolov5l.pt, cfg=, data=car-person.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2022-2-19 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, she